In [1]:
import os
import anndata
import logging
import numpy as np
import pandas as pd
from datetime import timedelta, datetime
import scanpy as sc
import seaborn as sns
import matplotlib.pyplot as plt
import leidenalg
import csv
from pandas import Timestamp
import re
import matplotlib.pyplot as plt
from scipy.stats import rankdata 
import scipy
import harmonypy as hm
import random
from scipy.stats.stats import pearsonr   
import scipy.sparse
from SCCAF import *

In [2]:
def correlation_analysis(scanpy_object, column_tosplitby, sum_or_mean):
    d = {}
    for cluster_number in np.unique(scanpy_object.obs[column_tosplitby].values):
        #cluster_number=np.unique(heart.obs['louvain_annotated'].values)[1]
        scanpy_object_subset = scanpy_object[scanpy_object.obs[column_tosplitby].isin([cluster_number])]

        if sum_or_mean=="mean":
            d[cluster_number] = np.squeeze(np.asarray(scanpy_object_subset.raw.X.mean(axis=0))) #np.log(+1)
        elif sum_or_mean=="sum":
            d[cluster_number] = np.squeeze(np.asarray(scanpy_object_subset.raw.X.sum(axis=0))) #np.log(+1)
        del scanpy_object_subset
        
    return d

In [7]:
path="/fast/AG_Huebner/huebner3/ANALYSES/20190926_gp_BadOyenhausen/scanpy/20210302/Global_touse/"

In [92]:
global_all=sc.read_h5ad(path + "global_all_ANNOTATED_CELLSTATES_RAW.h5ad")

In [93]:
global_all.obs['cell_type']=global_all.obs['cell_type']
subs = {
    "vEC":"EC",
    "nonvEC":"EC", 
    "SMC":"Mural", 
    "PC":"Mural", 
     } 
# PC-like, EC-like, MP-like, MP_interacting, CM-like, ECM_r, CST3+, ECM_p, Canonical, DAAM1+
x=[subs.get(item,item)  for item in global_all.obs['cell_type']]
global_all.obs['cell_type']=x
pd.value_counts(global_all.obs['cell_type'])/global_all.shape[0]

CM          0.358217
Mural       0.199124
FB          0.176731
EC          0.145108
Myeloid     0.079489
Lymphoid    0.022796
N           0.011461
Mast        0.003926
AD          0.003150
Name: cell_type, dtype: float64

In [95]:
#global_all.write(path_out + "global_all_ANNOTATED_CELLSTATES_RAW_V2.h5ad")

... storing 'cell_type' as categorical


In [90]:
global_all=sc.read_h5ad(path + "global_all_ANNOTATED_CELLSTATES_RAW_V2.h5ad")

In [96]:
global_all

AnnData object with n_obs × n_vars = 881081 × 33538
    obs: 'Patient', 'Sample', 'Gender', 'Gene', 'Diagnosis', 'Clinical.dominant.mutation', 'Age', 'Mutation.Type', 'Genomic.location', 'Region', 'Origin', 'n_genes', 'percent_mito', 'percent_ribo', 'X10X_version', 'n_counts', 'solo_score', 'predicted_doublets_solo', 'scrublet_score_z', 'predicted_doublets_z', 'scrublet_score_log', 'predicted_doublets_log', 'Replicate', 'cell_states_20200602', 'cell_type_20200602', 'cell_type_20200921', 'cell_type_global_HCA', 'cell_states_HCA', 'cell_type_fine_20200921', 'cell_states_20200921', 'cell_states_long_20200921', 'cell_states_20210218', 'validated_20210218', 'leiden_01', 'leiden_02', 'leiden_03', 'leiden_04', 'cell_type', 'cell_states', 'cell_states_long', 'validated', 'cell_type_toplot1', 'cell_type_toplot2', 'cell_type_toplot3', 'cell_type_toplot4'
    var: 'gene_ids', 'feature_types', 'genome'

In [97]:
#global_all=global_all[global_all.obs.Region.isin(['FW', 'AP', 'S'])].copy()
global_all=global_all[global_all.obs.Region.isin(['RV'])].copy()
global_all=global_all[global_all.obs.validated].copy()
global_all=global_all[global_all.obs['Clinical.dominant.mutation'].isin(['TTN', 'LMNA', 'RBM20', 'PKP2', 'control', 'PVneg'])].copy()

In [98]:
def paste0(i, j, k, l, m):
    return(str(i)+"__"+str(j)+"__"+str(k)+"__"+str(l)+"__"+str(m))


global_all.obs['pool_key']=[paste0(i,j,k,l,m) for i,j,k,l,m in zip(global_all.obs['Clinical.dominant.mutation'], 
                                                                            global_all.obs.cell_states, 
                                                                            global_all.obs.Patient,
                                                                            global_all.obs.X10X_version,
                                                                            global_all.obs.Gender
                                                                           )]

In [99]:
global_all#.obs['pool_key']

AnnData object with n_obs × n_vars = 182977 × 33538
    obs: 'Patient', 'Sample', 'Gender', 'Gene', 'Diagnosis', 'Clinical.dominant.mutation', 'Age', 'Mutation.Type', 'Genomic.location', 'Region', 'Origin', 'n_genes', 'percent_mito', 'percent_ribo', 'X10X_version', 'n_counts', 'solo_score', 'predicted_doublets_solo', 'scrublet_score_z', 'predicted_doublets_z', 'scrublet_score_log', 'predicted_doublets_log', 'Replicate', 'cell_states_20200602', 'cell_type_20200602', 'cell_type_20200921', 'cell_type_global_HCA', 'cell_states_HCA', 'cell_type_fine_20200921', 'cell_states_20200921', 'cell_states_long_20200921', 'cell_states_20210218', 'validated_20210218', 'leiden_01', 'leiden_02', 'leiden_03', 'leiden_04', 'cell_type', 'cell_states', 'cell_states_long', 'validated', 'cell_type_toplot1', 'cell_type_toplot2', 'cell_type_toplot3', 'cell_type_toplot4', 'pool_key'
    var: 'gene_ids', 'feature_types', 'genome'

In [100]:
global_all.raw=global_all
x=correlation_analysis(global_all, 'pool_key', 'sum')
x=pd.DataFrame(x)
x.index=global_all.var.index

In [101]:
# This is the count matrix for working with edgeR. The header contains most(ß) important covariates and will be used to create the metadata sheet
x.head()

,LMNA__AD1.0__H06__V3__f,LMNA__AD1.0__H24__V3__f,LMNA__AD1.0__H28__V3__m,LMNA__AD1.1__H06__V3__f,LMNA__AD1.1__H24__V3__f,LMNA__AD1.1__H25__V3__m,LMNA__AD1.1__H28__V3__m,LMNA__AD1.1__H29__V3__f,LMNA__AD2__H06__V3__f,LMNA__AD2__H24__V3__f,...,control__vFB4__BS_H37__V3__m,control__vFB4__D1__V2__f,control__vFB4__D2__V2__m,control__vFB4__D4__V2__f,control__vFB4__D5__V2__f,control__vFB4__D6__V2__m,control__vFB4__D7__V2__m,control__vFB4__H49__V3__f,control__vFB4__H53__V3__m,control__vFB4__H55__V3__m
MIR1302-2HG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
FAM138A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
OR4F5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AL627309.1,1.0,0.0,2.0,2.0,0.0,2.0,8.0,0.0,3.0,1.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,2.0,1.0,0.0
AL627309.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [102]:
x.to_csv( path_out + "ALL_CELLSTATE_PSEUDOBULK_RV.csv")

Filter genes:
<br>
We thought of filtering based on mean expression and library sized normalized counts, and not on raw counts as suggested by edgeR, as cutoffs will differ between cell-types (different abundance, different transcriptional activity...).

In [103]:
# UPDATED

In [104]:
global_all.obs['Gene_Patient']=[str(i) + "_" + str(j) + "_" + str(k) for i,j,k in zip(global_all.obs['Clinical.dominant.mutation'], 
                                                                                      global_all.obs.cell_states,
                                                                                      global_all.obs.Patient)]

In [105]:
subset_=sc.AnnData(global_all[:,global_all.var.index].raw.X, # 
                  obs=global_all.obs,
                  var=global_all[:,global_all.var.index].raw.var
                 )

In [106]:
subset_.raw=subset_
x=correlation_analysis(subset_, 'Gene_Patient', 'mean') # MEan per Gene_Celltype_Patient
x=pd.DataFrame(x)
x.index=global_all.var.index

In [107]:
# This table helps to remove genes
x

,LMNA_AD1.0_H06,LMNA_AD1.0_H24,LMNA_AD1.0_H28,LMNA_AD1.1_H06,LMNA_AD1.1_H24,LMNA_AD1.1_H25,LMNA_AD1.1_H28,LMNA_AD1.1_H29,LMNA_AD2_H06,LMNA_AD2_H24,...,control_vFB4_BS_H37,control_vFB4_D1,control_vFB4_D2,control_vFB4_D4,control_vFB4_D5,control_vFB4_D6,control_vFB4_D7,control_vFB4_H49,control_vFB4_H53,control_vFB4_H55
MIR1302-2HG,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0000,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
FAM138A,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0000,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
OR4F5,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0000,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
AL627309.1,0.071429,0.0,0.333333,0.076923,0.000000,0.153846,0.186047,0.0,0.1875,0.071429,...,0.0,0.0,0.000000,0.002309,0.0,0.021277,0.000000,0.027397,0.012821,0.0
AL627309.3,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0000,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AC233755.2,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0000,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
AC233755.1,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0000,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
AC240274.1,0.000000,0.0,0.000000,0.000000,0.076923,0.000000,0.000000,0.0,0.0000,0.000000,...,0.0,0.0,0.010417,0.000000,0.0,0.000000,0.043478,0.027397,0.025641,0.0
AC213203.1,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0000,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0


In [108]:
MUTATIONS=[str(x) + "_" + str(y) for x in global_all.obs['Clinical.dominant.mutation'].unique() for 
           y in global_all.obs['cell_states'].unique()]


In [109]:
LIST_COLNAMES=[]
for MUTATION in MUTATIONS:
    OUT=np.where([re.match(MUTATION, COLNAME) for COLNAME in x.columns])[0]
    LIST_COLNAMES.append(OUT)

x=[x.iloc[:,i].mean(axis=1) for i in LIST_COLNAMES] # Mean per Genotype_Cellstate
x=pd.concat(x, axis=1)

In [110]:
x.columns=MUTATIONS
x

,control_PC1,control_vCM1.3,control_SMC1.1,control_vCM1.0,control_vCM3.0,control_vCM2,control_vCM4,control_Mast,control_vFB1.0,control_EC1.0,...,PKP2_AD1.1,PKP2_AD1.0,PKP2_cDC1,PKP2_MY16,PKP2_Meso,PKP2_MY10,PKP2_prolif Lymphoids,PKP2_unclassfied.2,PKP2_NC1.3,PKP2_NC6
MIR1302-2HG,0.000000,0.000000,0.000000,0.000000,0.000023,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
FAM138A,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
OR4F5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
AL627309.1,0.004966,0.007135,0.003131,0.012439,0.024667,0.027263,0.020624,0.007937,0.007215,0.002436,...,0.071482,0.0,0.00,0.0,0.0,0.023060,0.0,0.0,0.044444,0.0
AL627309.3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AC233755.2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
AC233755.1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
AC240274.1,0.003098,0.001481,0.001907,0.001985,0.007750,0.005782,0.012960,0.001401,0.019618,0.001010,...,0.012246,0.0,0.04,0.0,0.0,0.043323,0.0,0.0,0.000000,0.0
AC213203.1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0


In [111]:
x.to_csv(path_out + "ALL_CELLSTATE_PSEUDOBULK_FILTERING_RV.csv")

In [75]:
x = pd.Series(global_all.obs.cell_type.values,
          index=global_all.obs.cell_states).to_dict()
x=pd.DataFrame.from_dict(x, orient='index')
x.head()

,0
SMC1.2,Mural
vCM2,CM
vCM1.0,CM
EC8.0,EC
PC1,Mural


In [77]:
x.to_csv(path_out + "CELLSTATE_TRANSLATION_TABLE.csv")

In [78]:
path_out + "CELLSTATE_TRANSLATION_TABLE.csv"

'/fast/AG_Huebner/huebner3/ANALYSES/20190926_gp_BadOyenhausen/scanpy/20210302/Global_touse/CELLSTATE_TRANSLATION_TABLE.csv'

# Pseudobulk for cell_type

In [16]:
global_all=sc.read_h5ad(path + "global_all_ANNOTATED_CELLSTATES_RAW_V2.h5ad")

In [17]:
global_all

AnnData object with n_obs × n_vars = 881081 × 33538
    obs: 'Patient', 'Sample', 'Gender', 'Gene', 'Diagnosis', 'Clinical.dominant.mutation', 'Age', 'Mutation.Type', 'Genomic.location', 'Region', 'Origin', 'n_genes', 'percent_mito', 'percent_ribo', 'X10X_version', 'n_counts', 'solo_score', 'predicted_doublets_solo', 'scrublet_score_z', 'predicted_doublets_z', 'scrublet_score_log', 'predicted_doublets_log', 'Replicate', 'cell_states_20200602', 'cell_type_20200602', 'cell_type_20200921', 'cell_type_global_HCA', 'cell_states_HCA', 'cell_type_fine_20200921', 'cell_states_20200921', 'cell_states_long_20200921', 'cell_states_20210218', 'validated_20210218', 'leiden_01', 'leiden_02', 'leiden_03', 'leiden_04', 'cell_type', 'cell_states', 'cell_states_long', 'validated', 'cell_type_toplot1', 'cell_type_toplot2', 'cell_type_toplot3', 'cell_type_toplot4'
    var: 'gene_ids', 'feature_types', 'genome'

In [18]:
#global_all=global_all[global_all.obs.Region.isin(['FW', 'AP', 'S'])].copy()
global_all=global_all[global_all.obs.Region.isin(['RV'])].copy()
global_all=global_all[global_all.obs.validated].copy()
global_all=global_all[global_all.obs['Clinical.dominant.mutation'].isin(['TTN', 'LMNA', 'RBM20', 'PKP2', 'control', 'PVneg'])].copy()

In [19]:
def paste0(i, j, k, l, m):
    return(str(i)+"__"+str(j)+"__"+str(k)+"__"+str(l)+"__"+str(m))


global_all.obs['pool_key']=[paste0(i,j,k,l,m) for i,j,k,l,m in zip(global_all.obs['Clinical.dominant.mutation'], 
                                                                            global_all.obs.cell_type, 
                                                                            global_all.obs.Patient,
                                                                            global_all.obs.X10X_version,
                                                                            global_all.obs.Gender
                                                                           )]

In [20]:
global_all#.obs['pool_key']

AnnData object with n_obs × n_vars = 182977 × 33538
    obs: 'Patient', 'Sample', 'Gender', 'Gene', 'Diagnosis', 'Clinical.dominant.mutation', 'Age', 'Mutation.Type', 'Genomic.location', 'Region', 'Origin', 'n_genes', 'percent_mito', 'percent_ribo', 'X10X_version', 'n_counts', 'solo_score', 'predicted_doublets_solo', 'scrublet_score_z', 'predicted_doublets_z', 'scrublet_score_log', 'predicted_doublets_log', 'Replicate', 'cell_states_20200602', 'cell_type_20200602', 'cell_type_20200921', 'cell_type_global_HCA', 'cell_states_HCA', 'cell_type_fine_20200921', 'cell_states_20200921', 'cell_states_long_20200921', 'cell_states_20210218', 'validated_20210218', 'leiden_01', 'leiden_02', 'leiden_03', 'leiden_04', 'cell_type', 'cell_states', 'cell_states_long', 'validated', 'cell_type_toplot1', 'cell_type_toplot2', 'cell_type_toplot3', 'cell_type_toplot4', 'pool_key'
    var: 'gene_ids', 'feature_types', 'genome'

In [21]:
global_all.raw=global_all
x=correlation_analysis(global_all, 'pool_key', 'sum')
x=pd.DataFrame(x)
x.index=global_all.var.index

In [22]:
# This is the count matrix for working with edgeR. The header contains most(ß) important covariates and will be used to create the metadata sheet
x.head()

,LMNA__AD__H06__V3__f,LMNA__AD__H24__V3__f,LMNA__AD__H25__V3__m,LMNA__AD__H27__V3__f,LMNA__AD__H28__V3__m,LMNA__AD__H29__V3__f,LMNA__CM__H06__V3__f,LMNA__CM__H24__V3__f,LMNA__CM__H25__V3__m,LMNA__CM__H27__V3__f,...,control__N__BS_H37__V3__m,control__N__D1__V2__f,control__N__D2__V2__m,control__N__D4__V2__f,control__N__D5__V2__f,control__N__D6__V2__m,control__N__D7__V2__m,control__N__H49__V3__f,control__N__H53__V3__m,control__N__H55__V3__m
MIR1302-2HG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
FAM138A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
OR4F5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AL627309.1,6.0,1.0,2.0,1.0,11.0,0.0,62.0,30.0,53.0,25.0,...,0.0,2.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0
AL627309.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
x.to_csv( path_out + "ALL_CELLTYPE_PSEUDOBULK_RV.csv")

# Abundance for cell_type

In [49]:
global_all=sc.read_h5ad(path + "global_all_ANNOTATED_CELLSTATES_RAW_V2.h5ad")

In [51]:
#global_all=global_all[global_all.obs.Region.isin(['FW', 'AP', 'S'])].copy()
global_all=global_all[global_all.obs.Region.isin(['RV'])].copy()
global_all=global_all[global_all.obs.validated].copy()
global_all=global_all[global_all.obs['Clinical.dominant.mutation'].isin(['TTN', 'LMNA', 'RBM20', 'PKP2', 'control', 'PVneg'])].copy()

In [52]:
x=pd.crosstab(global_all.obs['cell_type'],
            global_all.obs['Patient'])

y=pd.crosstab(global_all.obs['cell_states'],
            global_all.obs['Patient'])

In [53]:
x.to_csv( path_out + "ABSOLUTE_CELLTYPE_NUMBER_RV.csv")
y.to_csv( path_out + "ABSOLUTE_CELLSTATES_NUMBER_RV.csv")